# Ejercicio 1 - Modelado de datos como RDF
## Temática elegida
La temática que se ha escogido para modelar los datos ha sido la serie de animación japonesa "*Serial Experiments: Lain*", un anime vanguardista conocido por tocar temas filosóficos y de comunicación social (centrado en el comienzo de internet en los 90-2000).

Se han modelado principalmente **personajes, organizaciones y objetos** que aparecen en a serie, intentando hacer incapié en las similitudes de esta con elementos de la vida real.

Entre los diferentes **conceptos y entidades** modelados, podemos encontrar:

- **Personajes**: los personajes principales de la serie, de tipo Person
- **Lugares**: (Solamente uno, ya que la trama transcurre en Tokyo, Japón)
- **Organizaciones**: tomando el termino de organización como grupo organizado, como pueden ser empresas o sectas
- **Productos/Objetos**: muy generalmente, todos aquellos elementos desarrollados por alguien con un fin o propósito, desde internet a productos de compra venta. Por enumerar algunos:
    - Código de programación
    - Dispositivos electrónicos
    - Ordenadores
    - Redes de ordenadores
- **Otros conceptos que aparecen representados**:
    - Dios: de manera simplificada, teniendo en cuenta su papel en la serie (mejor definido en la ontología)
    - Videojuegos: muy simplificadamente
    - Páginas web: tanto como enlaces como entidades en sí, ya que juegan un papel dentro de la serie
    - Conceptos físicos: como resonancias magnéticas
    - Eventos 

Para modelar este dominio, se ha hecho uso de **elementos de otras ontologías**, principalmente de Wikidata, Schema.org y DBpedia, aunque también se han tenido en cuenta otras ontologías como la [VideoGameOntology](http://purl.org/net/VideoGameOntology#), [FOAF](http://xmlns.com/foaf/0.1/) o [GEO](http://www.w3.org/2003/01/geo/wgs84_pos#). Entre estos elementos encontramos:
- La entidad de *schema:Person*, que representa a una persona
- La entidad *schema:Organization*, que representa organizaciones
- Se ha usado *schema:City* para representar ciudades
- Diversos Qs de Wikidata, entre ellos *Q15836568* (Computer Network) o *Q1293220* (Resonancia de Schumman)
- Para los eventos se ha empleado *schema:Event*
- Para el videojuego mencionado en el grafdo, se ha empleado *vgo:GameGenre* para describir su género

No se han empleado ontologías que describan a los personajes como personajes de una serie animada ni como peronajes de ficción ya que se ha considerado que no era pertinente para la aplicación que se le pretenderá dar a los datos del grafo.

In [2]:
%pip install rdflib sparqlwrapper pyshex --quiet --user
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef

g = Graph()
base_path = "./lain.ttl"
g.parse(base_path)
g.serialize(format="turtle")

'@prefix dpedia: <http://dbpedia.org/resource/> .\n@prefix foaf: <http://xmlns.com/foaf/0.1/> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix schema1: <http://schema.org/> .\n@prefix uo: <http://uniovi.es/miw/uo283069/entity/> .\n@prefix uop: <http://uniovi.es/miw/uo283069/property/> .\n@prefix vgo: <http://purl.org/net/VideoGameOntology#> .\n@prefix wd: <http://www.wikidata.org/entity/> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\nuo:Afx-n-nPrv-View.CL a uo:ComputerCode ;\n    schema1:creator uo:Lain_Iwakura ;\n    schema1:name "Afx-n-nPrv-View.CL" ;\n    schema1:programmingLanguage wd:Q849146 ;\n    uop:references wd:Q312 ;\n    rdfs:comment "Afx-n-nPrv-View.CL is the source for a code walker." ;\n    foaf:image <https://lain.wiki/wiki/File:Walker.jpg> .\n\nuo:CRowView.CL a uo:ComputerCode ;\n    schema1:creator uo:Lain_Iwakura ;\n    schema1:

# Ejercicio 2 - Despliegue datos RDF + consultas SPARQL

- URL pública del endpoint de Blazergaph: http://156.35.95.43:9999/blazegraph/namespace/kb/sparql
- URL pública del endpoint de Apache Jena Fuseki: http://156.35.95.43:3030/SerialExperimentsLain/sparql

## Primera consulta: Familia
- Objetivo: Obtener los datos de personajes que formen una familia (padres, hijos o hermanos)
- Consulta NO FEDERADA
### Query:
```SPARQL
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?person ?personName (GROUP_CONCAT(DISTINCT ?relation; separator=", ") AS ?relations) (GROUP_CONCAT(DISTINCT ?relativeName; separator=", ") AS ?relatives)
WHERE {
  ?person schema:name ?personName .
  FILTER (LANG(?personName) = "en")

  {
    ?person schema:parent ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Parent" AS ?relation)
  }
  UNION
  {
    ?person schema:sibling ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Sibling" AS ?relation)
  }
  UNION
  {
    ?person schema:children ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Child" AS ?relation)
  }
}
GROUP BY ?person ?personName
ORDER BY ?personName ?relation

```

In [7]:
str_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?person ?personName (GROUP_CONCAT(DISTINCT ?relation; separator=", ") AS ?relations) (GROUP_CONCAT(DISTINCT ?relativeName; separator=", ") AS ?relatives)
WHERE {
  ?person schema:name ?personName .
  FILTER (LANG(?personName) = "en")

  {
    ?person schema:parent ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Parent" AS ?relation)
  }
  UNION
  {
    ?person schema:sibling ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Sibling" AS ?relation)
  }
  UNION
  {
    ?person schema:children ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Child" AS ?relation)
  }
}
GROUP BY ?person ?personName
ORDER BY ?personName ?relation
"""
query_result = g.query(str_query)

# Los resultados se recorren por filas, quizá te recuerde a resultsets de librerías para ejecutar consultas SQL.
# Cada fila contiene objeto de dominio de rdflib (URIRef, Literal...) en variables que se llaman exactamente como los nombres de variables en la consulta SPARQL
for a_row in query_result:
  print(f"Person: {a_row.person}, Name: {a_row.personName}, Relations: {a_row.relations}, Relatives: {a_row.relatives}")

Person: http://example.org/Lain_Iwakura, Name: Lain Iwakura, Relations: Parent, Sibling, Relatives: Yasuo Iwakura, Miho Iwakura, Mika Iwakura
Person: http://example.org/MihoIwakura, Name: Miho Iwakura, Relations: Child, Relatives: Mika Iwakura, Lain Iwakura
Person: http://example.org/MikaIwakura, Name: Mika Iwakura, Relations: Parent, Sibling, Relatives: Miho Iwakura, Yasuo Iwakura, Lain Iwakura
Person: http://example.org/YasuoIwakura, Name: Yasuo Iwakura, Relations: Child, Relatives: Mika Iwakura, Lain Iwakura


## Segunda consulta: Referencias


```SPARQL
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX ex: <http://example.org/>

SELECT ?object ?name ?subject
WHERE {
  ?object ex:references ?subject .
  ?object schema:name ?name . 
}
```

In [4]:
str_query ="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX ex: <http://example.org/>

SELECT ?object ?name ?subject
WHERE {
  ?object ex:references ?subject .
  ?object schema:name ?name . 
}
"""
query_result = g.query(str_query)

for a_row in query_result:
  print(f"Object: {a_row.object}, Name: {a_row.name}, Subject: {a_row.subject}")

Object: http://example.org/Protocol6, Name: Protocol 6, Subject: http://www.wikidata.org/entity/Q2551624
Object: http://example.org/Protocol7, Name: Protocol 7, Subject: http://www.wikidata.org/entity/Q28823658
Object: http://example.org/KnightsoftheEasternCalculus, Name: Knights of the Eastern Calculus, Subject: http://www.wikidata.org/entity/Q6422517
Object: http://example.org/CoplandOS, Name: Copland OS, Subject: http://www.wikidata.org/entity/Q1131259
Object: http://example.org/NaviV1, Name: Child's NAVI, Subject: http://www.wikidata.org/entity/Q2141790
Object: http://example.org/Navi2, Name: NAVI, Subject: http://www.wikidata.org/entity/Q306381
Object: http://example.org/Navi3, Name: HandiNAVI, Subject: http://www.wikidata.org/entity/Q420772
Object: http://example.org/TheWired, Name: The Wired, Subject: http://www.wikidata.org/entity/Q466
Object: http://example.org/KensingtonExperiment, Name: Kensington Experiment, Subject: http://www.wikidata.org/entity/Q496036
Object: http://exa

## Tercera consulta: Federada
- Objetivo: Obtener, en base a la consulta anterior, los nombres de los sujetos en Wikidata

In [2]:
str_query ="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX ex: <http://example.org/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT ?object ?name ?subject ?subjectLabel
WHERE {
    ?object ex:references ?subject .
    ?object schema:name ?name . 
    
    SERVICE <https://query.wikidata.org/sparql> {
        ?subject rdfs:label ?subjectLabel .
        FILTER (LANG(?subjectLabel) = "en")
    }
}
"""
query_result = g.query(str_query)

for a_row in query_result:
  print(f"Object: {a_row.object}, Name: {a_row.name}, Subject: {a_row.subjectLabel} - {a_row.subject}")

NameError: name 'g' is not defined

## Cuarta consula: Relaciones entre nodos

- Objetivo (un poco complejo): Comprobar si hay o no una relación entre dos entidades, para ello utilizaremos el oeprador del wildcard <>.
- No federada

In [67]:
"""
Explicación:
En cualquier tripleta, el predicado tiene que obligatoriamente ser <> o no serlo,
es decir (<>|!<>), si además a esto le añadimos el operador de relación inversa (^),
podemos decir que, en una relación entre dos entidades que estén conectadas, debe existir
una relación tal que (<>|!<>)|^(<>|!<>). Esto sumado al operador * que indica que 
la relación puede ser de cualquier longitud, nos permite saber si dos entidades están
conectadas o no. (En este caso, como el grafo es epqueño y todo está conectado, 
siempre va a haber una conexión entre dos entidades por muy larga que sea).
"""
ask_query = """
ASK {
    ex:MenInBlack ((<>|!<>)|^(<>|!<>))* ex:God
}
"""

ask_result = g.query(ask_query)

if ask_result.askAnswer:
    print("There is a connection between MenInBlack and God.")
else:
    print("There is no connection between MenInBlack and God.")

There is a connection between MenInBlack and God.


In [10]:
query = """
PREFIX schema: <http://schema.org/>

SELECT ?japaneseName ?personName ?occupationLabel ?placeOfOccupation
WHERE {
    ?person a schema:Person  .
    OPTIONAL { ?person schema:name ?japaneseName . FILTER (LANG(?japaneseName) = "ja") }
    ?person schema:name ?personName .
    ?person schema:hasOccupation ?occupation .
    ?occupation a schema:Occupation .
    ?occupation schema:name ?occupationLabel .
    OPTIONAL { ?person schema:employmentUnit ?placeOfOccupation }

    FILTER (LANG(?personName) = "en")
}
"""

query_result = g.query(query)

for a_row in query_result:
    print(f"Japanese Name: {a_row.japaneseName}, English Name: {a_row.personName}, Occupation: {a_row.occupationLabel}, Place of Occupation: {a_row.placeOfOccupation}\n")

Japanese Name: 岩倉 康男, English Name: Yasuo Iwakura, Occupation: Computer Programmer, Place of Occupation: http://example.org/TachibanaLabs

Japanese Name: 英利 政美, English Name: Masami Eiri, Occupation: Computer Programmer, Place of Occupation: http://example.org/TachibanaLabs

Japanese Name: None, English Name: Professor Hodgeson, Occupation: Scientist, Place of Occupation: None

Japanese Name: None, English Name: Karl Haushofer, Occupation: Spy, Place of Occupation: None

Japanese Name: None, English Name: Lin Sui-Xi, Occupation: Spy, Place of Occupation: None



In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")  # Construimos un objeto para acceder a cierto endpoint. En este caso, DBpedia
sparql.setQuery("""

prefix wdt: 
prefix wd: 
prefix wikibase: 
prefix dbr: 
prefix dbo: 
prefix dbp: 
prefix ex:<http://156.35.95.43:3030/SerialExperimentsLain/sparql>

                
SELECT ?wikidataItem ?name ?subjectLabel
  SERVICE {
      ?subject ex:references ?wikidataItem .
      ?subject rdfs:label ?subjectLabel .
      FILTER (LANG(?subjectLabel) = "en")
  }
"""
)

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    for a_var_key in result:
      print(result[a_var_key]["value"], end="       ")
    print()
print("--------")

HTTPError: HTTP Error 403: Forbidden

In [14]:
query = """
PREFIX ex: <http://example.org/>
PREFIX schema: <http://schema.org/>
PREFIX dpedia: <http://dbpedia.org/resource/>

SELECT (COUNT(?object) AS ?count) 
WHERE {
  ?object a ?type .
  VALUES ?type { ex:ComputerCode schema:WebPage schema:VideoGame dpedia:Electrical_device dpedia:Computer_network dpedia:Computer dpedia:Operating_system }
}
"""

query_result = g.query(query)

for a_row in query_result:
    print(f"Number of objects: {a_row['count']}")

Number of objects: 9


In [ ]:
# Hacer algo como FILTER(?bdate >= "1960-01-01"^^xsd:date)

# Ejercicio 3: Desarrollo de aplicación web 

Url de la web: 

## Descripción del contenido

Se ha tratado de ofrecer al usuario una forma "fácil" de navegar a través del grafo de RDF. La intención de la web es permitir al usuario visualizar todos los nodos **pertinentes** del grafo mostrandle la información de todas sus relaciones (exceptuando la de rdf:type).

El contenido de la web se basa en la serie japonesa escogida para modelar los datos, y sigue la siguiente lógica.

![Diagrama en resources/Web.png](/resources/Web.png)
                                          
## Descripción de la forma de uso de RDFa
Se ha empleado RDFa para describir los nodos representados en la pantalla del usuario.
// TODO

# Ejercicio 4 - Validación datos RDF

Los ficheros utilizados se encuentran bajo la carpeta SHEX, dentro de ella hay:
- CharacterShape: shape para personajes
- LocationShape: shpae para lugaress
- OccupationShape: shape para ocupación (trabajo)
- OrganizationShape: shape para organizaciones
- ComputerCodeShape: shape para programas informáticos
- SHapeMap.sm: EL shape map utilzado para validar los datos

In [ ]:
from pyshex import ShExEvaluator
from pyshex.evaluate import evaluate

with open("SHEX/FullSHape.shex", "r") as file:
    shapes = file.read()

evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                          focus="http://uniovi.es/miw/uo283069/entity/Lain_Iwakura", 
                          start="SchemaCharacter")
results = evaluator.evaluate()

for result in results:
    print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")



SPARQL'''
PREFIX schema: <http://schema.org/>
select ?node where{
	?node a schema:Person .
	FILTER(isIRI(?node))
}
'''@<SchemaCharacter>,
 does not look like a valid URI, trying to serialize this will break.


Node: 
SPARQL'''
PREFIX schema: <http://schema.org/>
select ?node where{
	?node a schema:Person .
	FILTER(isIRI(?node))
}
'''@<SchemaCharacter>,
, Conforms: False, Reason: Focus: 
SPARQL'''
PREFIX schema: <http://schema.org/>
select ?node where{
	?node a schema:Person .
	FILTER(isIRI(?node))
}
'''@<SchemaCharacter>,
 not in graph


In [7]:
evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                          focus="http://uniovi.es/miw/uo283069/entity/Tokyo", 
                          start="SchemaLocation")
results = evaluator.evaluate()

for result in results:
    print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")

Node: http://uniovi.es/miw/uo283069/entity/Tokyo, Conforms: True, Reason: 


In [ ]:
evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                          focus="http://uniovi.es/miw/uo283069/entity/TachibanaLabs", 
                          start="SchemaOrganization")
results = evaluator.evaluate()

for result in results:
    print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")

Node: http://uniovi.es/miw/uo283069/entity/TachibanaLabs, Conforms: True, Reason: 


In [14]:

nodes = ["http://uniovi.es/miw/uo283069/entity/CRowView.CL", "http://uniovi.es/miw/uo283069/entity/Afx-n-nPrv-View.CL"]

for node in nodes:
    evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                              focus=node, 
                              start="ComputerCodeShape")
    results = evaluator.evaluate()

    for result in results:
        print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")

Node: http://uniovi.es/miw/uo283069/entity/Afx-n-nPrv-View.CL, Conforms: True, Reason: 


### Shape map

## Ejercicio : Extracción de shapes

In [1]:
%pip install shexer --quiet

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'c:\\Python311\\Scripts\\flask.exe' -> 'c:\\Python311\\Scripts\\flask.exe.deleteme'

